The "Broken" Weather Station
Background: A mountain weather station has been offline for months and recently sent a burst of messy data. Before we can use the data and connect it to streamflow, it must be repaired and the Pandas toolkit can support this.

Task 1: The Raw Feed
Load the .csv of rainfall values and convert into a Pandas DataFrame named Rainfall_mm. The data can be found here:

data/snotel_rainfall_data.csv

Check your work: Use .head() and .describe(). Does the data look right?

In [1]:
import pandas as pd

# The "Broken" Weather Station

Background: A mountain weather station has been offline for months and recently sent a burst of messy data. Before we can use the data and connect it to streamflow, it must be repaired and the Pandas toolkit can support this.

## Task 1: The Raw Feed 
Load the .csv of rainfall values and convert into a Pandas DataFrame named Rainfall_mm. The data can be found here:

```data/snotel_rainfall_data.csv```

Check your work: Use .head() and .describe(). Does the data look right?


In [ ]:
# Load rainfall data
Rainfall_mm = pd.read_csv("data/snotel_rainfall_data.csv") # Load the dataset

Rainfall_mm.head(3)

Rainfall_mm.describe()

,Date,Precip_in
count,61,58
unique,60,32
top,2024-01-16,0.0
freq,2,27


In [16]:
Rainfall_mm.head(20)

,Date,Precip_in
0,2024-01-01,0.14028030062619642
1,2024-01-02,0.6590180328421851
2,2024-01-03,0.0
3,2024-01-04,0.0
4,2024-01-05,0.3757291395473922
5,2024-01-06,0.0
6,2024-01-07,1.0954577625968716
7,2024-01-08,0.0
8,2024-01-09,0.0
9,2024-01-10,0.18657649912973773


## Task 2: Using Pandas Tools to make the data readable

**coerce** turns anything it can't read as a number (like 'T' or 'error') into NaN


```df['colname'] = pd.to_numeric(df['colname'], errors='coerce')```

Apply this to your dataset and then try .describe()

In [30]:
Rainfall_mm['Precip_in'] = pd.to_numeric(Rainfall_mm['Precip_in'], errors='coerce') 
Rainfall_mm.describe()
# remove values greater than 20 and less than 0 from the Precip_in column
Rainfall_mm_cleaned = Rainfall_mm[(Rainfall_mm['Precip_in'] <= 20) & (Rainfall_mm['Precip_in'] >= 0)]
Rainfall_mm_cleaned.describe()

,Precip_in
count,54.000000
mean,0.460056
std,0.790430
min,0.000000
25%,0.000000
50%,0.050563
75%,0.641326
max,3.609354


## Task 3: Load the discharge data

Load the .csv of rainfall values and convert into a Pandas Dataframe named streamflow_cfs. The data can be found here:

```data/streamflow_data.csv```

Check your work: Use .head() and .describe(). Does the data look right? How can you extract more useful statistics? Show this.

In [27]:
streamflow_cfs = pd.read_csv("data/streamflow_data.csv")
streamflow_cfs.describe()
streamflow_cfs.head(20)
streamflow_cfs['Streamflow_cfs'] = pd.to_numeric(streamflow_cfs['Streamflow_cfs'], errors='coerce') 
streamflow_cfs.describe()



,Streamflow_cfs
count,54.000000
mean,40.058333
std,369.096212
min,-999.000000
25%,10.100000
50%,10.600000
75%,13.900000
max,2510.000000


## Task 4: Clean and repair the data

The sensor cut out during the storm and we now have NaN **AND** -999 values. This prevents us from plotting or otherwise using the time series. Are there any other values we should remove?

Explore different methods and select one that fits the data.

Explore .dropna() (delete the gap), .fillna() with the mean, and .interpolate() for a smoother hydrograph for both datasets.

Note, it may be useful to create a new Pandas DataFrame to compare differences. 

In [39]:
# remove NaN and -999 values from the Streamflow_cfs column
streamflow_cfs_cleaned = streamflow_cfs[(streamflow_cfs['Streamflow_cfs'] >= 0) & (streamflow_cfs['Streamflow_cfs'].notna())]
streamflow_cfs_cleaned.describe()   
# create a data set replacing the removed values with the mean of the column
streamflow_cfs_filled = streamflow_cfs.copy()
streamflow_cfs_filled['Streamflow_cfs'] = streamflow_cfs_filled['Streamflow_cfs'].replace(-999, streamflow_cfs_filled['Streamflow_cfs'].mean())
streamflow_cfs_filled['Streamflow_cfs'] = streamflow_cfs_filled['Streamflow_cfs'].fillna(streamflow_cfs_filled['Streamflow_cfs'].mean())
streamflow_cfs_filled.describe()        
# create a data set interpolating the removed values
streamflow_cfs_interpolated = streamflow_cfs.copy()
streamflow_cfs_interpolated['Streamflow_cfs'] = streamflow_cfs_interpolated['Streamflow_cfs'].replace(-999, pd.NA)
streamflow_cfs_interpolated['Streamflow_cfs'] = streamflow_cfs_interpolated['Streamflow_cfs'].interpolate(method='linear')
streamflow_cfs_interpolated.describe()  


/scratch/local/u1589214/845786/ipykernel_3279183/2806379309.py:12: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  streamflow_cfs_interpolated['Streamflow_cfs'] = streamflow_cfs_interpolated['Streamflow_cfs'].interpolate(method='linear')


,Date,Streamflow_cfs
count,60,53.0
unique,60,35.0
top,2024-01-01,10.0
freq,1,8.0


## Task 5: Join Pandas DataFrames

We often want to relate data to another, and having all the data in once centralized data frame supports this comparison. Create a new Pandas DataFrame named:
* rainfall_methods fill it with the three rainfall dataframes we cleaned
* streamflow_methods and fill it with the three streamflow dataframes we cleaned


In [40]:
# create merged dataset with the cleaned, filled, and interpolated streamflow data
merged_data = pd.merge(Rainfall_mm_cleaned, streamflow_cfs_cleaned, on='Date', how='inner')
merged_data_filled = pd.merge(Rainfall_mm_cleaned, streamflow_cfs_filled, on='Date', how='inner')
merged_data_interpolated = pd.merge(Rainfall_mm_cleaned, streamflow_cfs_interpolated, on='Date', how='inner')           
merged_data.describe()


,Precip_in,Streamflow_cfs
count,51.000000,51.000000
mean,0.458521,61.533333
std,0.806482,349.729992
min,0.000000,10.000000
25%,0.000000,10.100000
50%,0.000000,10.700000
75%,0.623634,13.950000
max,3.609354,2510.000000


## Task 6: Plot the rainfall and streamflow data to visualize trends and relationships.

Use Pandas simple plotting functionality to separately plot the two dataframes. Which interpolation method do you like? 

In [41]:
# plot the cleaned, filled, and interpolated streamflow data against the cleaned rainfall data
import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.


## Task 7: Combining DataFrames

From the plots above, choose your most representative gap filling dataframe for rainfall and streamflow, and combine them into a rain_flow_df DataFrame.

## Task 8: Calculate the monthly statistics of the rainfall and discharge

Calculate the monthly total, mean daily (for each month), and the maximum (for each month) rainfall and flow. 

## Task 9: Data Corrections

We want to put the data into a streamflow model, but it requires precipitation to be in mm and streamflow to be in CMS (cubic meters per second). Create a new dataframe called rain_flow_SI_df that converts the previous dataframe to SI units.

## Task 10: Event diagnostics

A key element of hydroinformatics is to identify key events and learn from them. Here, we have two tasks.
* Create a new column and programatically label each day as 'Dry', 'Light Rain', or 'Heavy Rain' based on the precipitaiton colum.
* From our rain_flow_SI_df, create a new Pandas DataFrame called storm_df that programatically selects the streamflow and precipitation data for 5 days before and after the peak flow event.

## Task 11: Quick Data Visualziation

Use the pandas plot function to conduct a quick visualization of precipitation and streamflow. Do they seem coorelated? Any glaring errors?